##### DNN 기반 회귀 모델 구현 + 학습 진행 모니터링
- 데이터셋   : iris.csv
- 피쳐/속성  : 3개 Sepal_Length, Sepal_width, Petal_Width
- 타겟/라벨  : 1개 Petal_
- 학습/방법  : 지도학습 -> 회귀
- 알고리즘   : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch

***

- 모니터링
    * 기준 : 검증데이터셋의 loss 또는 score
    * 평가 : 학습데이터셋의 loss 또는 score와 비교해서 학습 중단여부 결정
    * 선택 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장

[1] 모듈 로딩 및 데이터 준비 <hr>

In [16]:
# 모델 관련 모듈 로딩
import torch
import torch.nn as nn  
import torch.nn.functional as F  
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 관련 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [17]:
# 활용 패키지 버전 체크
print(f'torch V.:{torch.__version__}')
print(f'pandas V.:{pd.__version__}')

torch V.:2.4.1
pandas V.:2.0.3


In [18]:
## 데이터 로딩
DATA_FILE='../Data/iris.csv'

### CSV => DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

### 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스목적 : iris 데이터를 학습 및 추론
- 클래스이름 : irisRegModel
- 부모클래스 : nn.Module
- 매개 변수 : 층별 입출력 개수 고정하기 때문에 필요 x!
- 속성 필드 : 
- 기능 역할 : __ init__() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding) - [상속 시 가능!]
- 클래스구조
    * 입력층 : 입력 3개(피쳐)  출력 10개 (퍼셉트론/뉴런 10개 존재)
    * 은닉층 : 입력 10개       출력 30개 (퍼셉트론/뉴런 30개 존재)
    * 출력층 : 입력 30개       출력 1개 (너비값)

- 활성화함수
    * 클래스 형태 ==> ex) nn.MESLoss, nn.ReLU ==> __ init__(self) 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward(self) 메서드

In [19]:
class irisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        
        self.in_layer=nn.Linear(3, 10)
        self.hidden_layer=nn.Linear(10, 30)
        self.out_layer=nn.Linear(30, 1)

    # 순방향 학습 진행 메서드
    def forward(self, x):
        # - 입력층
        y = self.in_layer(x)     # y = f1w1 + f2w2 + f3w3 + b ... -> 10개
        y = F.relu(y)            # relu -> y 값의 범위 0 <= y
        
        # - 은닉층 : 10개의 숫자 값(>=0)
        y = self.hidden_layer(y) # y = f21w21 + ... + f210w210 , ... -> 30개
        y = F.relu(y)            # relu -> y 값의 범위 0 <= y

        # - 출력층 : 1개의 숫자 값(>=0)
        return self.out_layer(y)        # f31w31 + ... f330w330 + b -> 1개

In [20]:
## 모델 인스턴스 생성
model = irisRegModel()
print(model)

irisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [21]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(1000,3)) # input_size = ,feature 개수)

Layer (type:depth-idx)                   Output Shape              Param #
irisRegModel                             [1000, 1]                 --
├─Linear: 1-1                            [1000, 10]                40
├─Linear: 1-2                            [1000, 30]                330
├─Linear: 1-3                            [1000, 1]                 31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.40
Input size (MB): 0.01
Forward/backward pass size (MB): 0.33
Params size (MB): 0.00
Estimated Total Size (MB): 0.34

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐_개수 : 3개
- 타겟_개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성_필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드 : 
    * __ init__(self): 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * __ len__(self): 데이터의 개수 반환
    * __ getitem__(self, index): 특정 인덱스의 피쳐와 타겟 반환
    

In [22]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [23]:
## 데이터셋 인스턴스 생성

# DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]     # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]]      # 2D (150, 1)

# - 커스텀데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF, targetDF)

[4] 학습 준비
- 학습 횟수 : EPOCH          <- 처음~ 끝까지 공부하는 단위
- 배치 크기 : BATCH_SIZE     <- 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE         <- 탠서 저장 및 실행 위치 (GPU/CPU)
- 학 습 률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1 (낮을수록 촘촘히)

In [24]:
### 학습 진행 관련 설정
EPOCHS = 1000
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0]//BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

print(f'BATCH_CNT: {BATCH_CNT}')

BATCH_CNT: 15


- 인스턴스/객체 : 모델, 데이터셋, 최적화, (손실함수), (성능지표)

In [25]:
# 모델 인스턴스 생성
model=irisRegModel()

# 데이터셋 인스턴스 생성
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'[X_train(shape): {X_train.shape} (type): {type(X_train)}], X_test: {X_test.shape}, X_val: {X_val.shape}')
print(f'[y_train(shape): {y_train.shape} (type): {type(y_train)}], y_test: {y_test.shape}, y_val: {y_val.shape}')

trainDS=IrisDataset(X_train, y_train)
valDS=IrisDataset(X_val, y_val)
testDS=IrisDataset(X_test, y_test)

# 데이터로더 인스턴스 생성
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

[X_train(shape): (84, 3) (type): <class 'pandas.core.frame.DataFrame'>], X_test: (38, 3), X_val: (28, 3)
[y_train(shape): (84, 1) (type): <class 'pandas.core.frame.DataFrame'>], y_test: (38, 1), y_val: (28, 1)


In [26]:
## [테스트] 데이터로더
for feature, target in trainDL:
    # print(feature, target)
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [27]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀 : MSE, MAE, RMSE, ...
reg_loss = nn.MSELoss()

[5] 학습 진행

In [28]:
# 학습 모드 함수
def training():
    # 학습 모드로 모델 설정
    model.train()
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0
    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y=model(featureTS)

        # 손실 계산
        loss=reg_loss(pre_y, targetTS)
        loss_total+=loss.item()
        
        # 성능평가 계산
        score=R2Score()(pre_y, targetTS)
        score_total+=score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss_total, score_total

In [29]:
# 검증 모드 함수
def validate():
    # 검증 모드로 모델 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
        # 평가
        pre_val=model(val_featureTS)

        # 손실
        loss_val=reg_loss(pre_val, val_targetTS)

        # 성능평가
        score_val=R2Score()(pre_val, val_targetTS)
    return loss_val, score_val

In [15]:
## 학습의 효과 확인 => 손실값과 성능평가값 저장 필요
loss_history, score_history=[[],[]], [[],[]] # train, val


## 학습 모니터링/스케쥴링 설정
# => loss_history, score_history 활용
# => 임계기준 : 10번
BREAK_CNT = 0
THESHOLD = 9
for epoch in range(1, EPOCHS):
    # 학습 모드 함수
    loss_total, score_total = training()

    # 검증 모드 함수
    loss_val, score_val = validate()

    # 에포크당 손실값과 성능평가값 저장
    loss_history[0].append(loss_total/epoch)
    score_history[0].append(score_total/epoch)

    loss_history[1].append(loss_val)
    score_history[1].append(score_val)

    # 학습 진행 모니터링/스케쥴링: - 검증 DS 기준

    # Loss 기준
    if len(loss_history[1]) >= 2:
        if loss_history[1][-1] >= loss_history[1][-2]: BREAK_CNT += 1
        
    # # score 기준
    # if len(score_history[1]) >= 2:
    #     if score_history[1][-1] <= score_history[1][-2]: BREAK_CNT += 1

    # 학습 중단 여부 설정
    if BREAK_CNT >= THESHOLD:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break

    print(f'{epoch}/{EPOCHS} => [TRAIN] LOSS: {loss_history[0][-1]} SCORE: {score_history[0][-1]}')
    print(f'\t=> [VAL] LOSS: {loss_history[1][-1]} SCORE: {score_history[1][-1]}')

1/100 => [TRAIN] LOSS: 22.699562072753906 SCORE: -39.613712310791016
	=> [VAL] LOSS: 2.008556604385376 SCORE: -2.111578941345215
2/100 => [TRAIN] LOSS: 9.651025891304016 SCORE: -16.19494390487671
	=> [VAL] LOSS: 1.694421648979187 SCORE: -1.6249330043792725
3/100 => [TRAIN] LOSS: 5.3264808257420855 SCORE: -8.431118647257486
	=> [VAL] LOSS: 1.4036709070205688 SCORE: -1.1745131015777588
4/100 => [TRAIN] LOSS: 3.1983697563409805 SCORE: -4.608752429485321
	=> [VAL] LOSS: 1.1491702795028687 SCORE: -0.7802506685256958
5/100 => [TRAIN] LOSS: 2.005863916873932 SCORE: -2.4807134866714478
	=> [VAL] LOSS: 0.9638012051582336 SCORE: -0.4930838346481323
6/100 => [TRAIN] LOSS: 1.321322739124298 SCORE: -1.2876572012901306
	=> [VAL] LOSS: 0.8563657999038696 SCORE: -0.32664918899536133
7/100 => [TRAIN] LOSS: 0.941074516092028 SCORE: -0.6640459469386509
	=> [VAL] LOSS: 0.8170455098152161 SCORE: -0.2657356262207031
8/100 => [TRAIN] LOSS: 0.7333230152726173 SCORE: -0.36562711000442505
	=> [VAL] LOSS: 0.8048

[테스트 & 검증 상태]
- 설정된 W,b 검증 및 테스트용 데이터셋 예측값 추출
- 사용되지 않는 기능들 OFF
- W, b 업데이트 X
    * -> 기능 OFF Auto_grade 엔진 ---> model.eval()
    * -> W, b 텐서 required_grade=True ---> no.grad()

In [102]:
# 테스트 진행
model.eval()
with torch.no_grad():
    # 테스트 데이터셋
    test_featureTS=torch.FloatTensor(testDS.featureDF.values)
    test_targetTS=torch.FloatTensor(testDS.targetDF.values)

    # 평가
    pre_test=model(test_featureTS)

    # 손실
    loss_test=reg_loss(pre_test, test_targetTS)

    # 성능평가
    score_test=R2Score()(pre_test, test_targetTS)
print(f'[TEST] LOSS: {loss_test} \n\tSCORE: {score_test}')

[TEST] LOSS: 0.04896678775548935 
	SCORE: 0.9080376625061035


- 모델 저장 방법 <hr>
- 방법1 : 모델 파라미터만 저장
- 방법2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [31]:
### 학습된 모델 파라미터 값 확인

model.state_dict()

- [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [57]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH='../Models/iris/'

# 저장 파일명
SAVE_FILE='model_train_wb.pth'

In [59]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)  # 폴더 / 폴더 / ... 하위 폴더까지 생성

In [60]:
# 모델 저장
torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)

In [54]:
# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
# [2] 모델의 state_dict 속성에 저장

# 읽기
wbTS = torch.load(SAVE_FILE)

C:\Users\KDP-50\AppData\Local\Temp\ipykernel_15464\1767297501.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wbTS = torch.load(SAVE_FILE)


In [61]:
# 모델 인스턴스에 저장
model.load_state_dict(wbTS)

<All keys matched successfully>

In [62]:
# 모델 인스턴스에 저장
model2 = irisRegModel() # 층마다 W,b 초기화
model2.load_state_dict(wbTS)

<All keys matched successfully>

In [63]:
print(model2)

irisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
